# Generative Agents in LangChain

This notebook implements a generative agent based on the paper [Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442) by Park, et. al.

In it, we leverage a time-weighted Memory object backed by a LangChain Retriever.

In [1]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from termcolor import colored

In [3]:
USER_NAME = "Pam"  # The name you want to use when interviewing the agent.

# Local LLM
LLM = ChatOllama(
    model="phi3",
    keep_alive=-1, # keep the model loaded indefinitely
    temperature=0,
    max_new_tokens=4096 # officailly 4096
    )

### Generative Agent Memory Components

This tutorial highlights the memory of generative agents and its impact on their behavior. The memory varies from standard LangChain Chat memory in two aspects:

1. **Memory Formation**

   Generative Agents have extended memories, stored in a single stream:
      1. Observations - from dialogues or interactions with the virtual world, about self or others
      2. Reflections - resurfaced and summarized core memories


2. **Memory Recall**

   Memories are retrieved using a weighted sum of salience, recency, and importance.

You can review the definitions of the `GenerativeAgent` and `GenerativeAgentMemory` in the [reference documentation]("https://api.python.langchain.com/en/latest/modules/experimental.html") for the following imports, focusing on `add_memory` and `summarize_related_memories` methods.

In [4]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

## Memory Lifecycle

Summarizing the key methods in the above: `add_memory` and `summarize_related_memories`.

When an agent makes an observation, it stores the memory:
    
1. Language model scores the memory's importance (1 for mundane, 10 for poignant)
2. Observation and importance are stored within a document by TimeWeightedVectorStoreRetriever, with a `last_accessed_time`.

When an agent responds to an observation:

1. Generates query(s) for retriever, which fetches documents based on salience, recency, and importance.
2. Summarizes the retrieved information
3. Updates the `last_accessed_time` for the used documents.


## Create a Generative Character



Now that we've walked through the definition, we will create two characters named "Tommie" and "Eve".

In [5]:
import numpy as np

import faiss


def score_normalizer(val: float) -> float:
    ret = 1.0 - 1.0 / (1.0 + np.exp(val))
    #print("val: "+str(float(val))+"_"+"ret: "+str(ret))
    return ret

def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OllamaEmbeddings(model="phi3")

    # Automatically determine the size of the embeddings
    test_embedding = embeddings_model.embed_query("test query")
    embedding_size = len(test_embedding)
    
    # Initialize the vectorstore as empty
    index = faiss.IndexFlatL2(embedding_size)
    
    # Initialize FAISS vector store
    vectorstore = FAISS(
        embeddings_model,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=score_normalizer,
        normalize_L2=True
    )
    
    # Create and return the retriever
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, 
        other_score_keys=["importance"], 
        k=15
    )

In [6]:
tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

tommie = GenerativeAgent(
    name="Alex",
    age=25,
    traits=" innovative, analytical, urban planner, environmentalist",  # You can add more persistent traits here
    status="interview",  # When connected to a virtual world, we can have the characters update their status
    llm=LLM,
    memory=tommies_memory,
)

In [7]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(tommie.get_summary())

Name: Alex (age: 25)
Innate traits:  innovative, analytical, urban planner, environmentalist
Alex is a diligent and detail-oriented individual who values accuracy and thoroughness in their work. They are also adaptable, capable of adjusting to new situations or challenges with ease. Alex demonstrates strong problem-solving skills and exhibits patience when dealing with complex issues. Additionally, they possess effective communication abilities, ensuring clear understanding among team members. Overall, Alex is a reliable and collaborative professional who prioritizes quality in their endeavors.


In [8]:
# We can add memories directly to the memory object
tommie_observations = [
    "Alex recalls the time when they led a project to convert an abandoned lot into a community garden, which became a popular local hangout."
    "Alex is passionate about integrating green spaces into urban environments, inspired by a childhood spent in nature."
    "Alex remembers the success of their last sustainable housing project, which reduced energy consumption by 30%."
    "Alex feels strongly about reducing the carbon footprint of cities, motivated by reports of rising global temperatures."
    "Alex recalls a conversation with a colleague about the importance of public transportation in reducing urban congestion."
    "Alex remembers attending a workshop on the benefits of vertical gardens in urban settings."
    "Alex is influenced by a mentor who emphasized the importance of community involvement in urban planning."
    "Alex recalls the challenges faced when advocating for bicycle lanes in a car-centric city."
    "Alex remembers a successful campaign to install solar panels on municipal buildings."
    "Alex feels inspired by a trip to a European city known for its sustainable urban practices."
]

for observation in tommie_observations:
    tommie.memory.add_memory(observation)

In [9]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(tommie.get_summary(force_refresh=True))

Name: Alex (age: 25)
Innate traits:  innovative, analytical, urban planner, environmentalist
Alex is an environmentally conscious individual with a strong commitment to integrating green spaces into urban landscapes, driven by personal experiences and childhood memories of nature. They have demonstrated leadership in projects that promote sustainability, such as converting abandoned lots into community gardens and implementing energy-efficient housing designs. Alex values the reduction of carbon emissions and is motivated by global environmental concerns. Their advocacy extends to public transportation's role in alleviating urban congestion and promoting vertical gardening for city environments. Community engagement is a cornerstone of their approach, as they learned from mentors who prioritize it in urban planning. Despite facing resistance, Alex has successfully championed initiatives like bicycle lanes and solar energy installations on public buildings. Their passion was further ign

## Pre-Interview with Character

Before sending our character on their way, let's ask them a few questions.

In [10]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [11]:
interview_agent(tommie, "What are you looking forward to doing today?")

/Users/wexzhang/anaconda3/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'Alex said "I\'m actually reviewing some plans for a new urban green space initiative that I believe could really benefit our community. It involves creating more accessible parks and integrating vertical gardens into existing structures. How about you? Any exciting projects or activities on your agenda today?"'

In [12]:
interview_agent(tommie, "What are you most worried about today?")

'Alex said "I\'m always mindful of potential challenges in implementing these green initiatives, like funding and public acceptance. But seeing our community come together for projects that improve our environment gives me hope. What about you? Any concerns on your agenda today?"'

## Step through the day's observations.

In [13]:
# Let's have Tommie start going through a day in the life.
observations = [
    "Alex looks at the city's current transportation map and notices the lack of bike lanes, thinking about ways to integrate them into the existing infrastructure."
    "Alex sees an article about a new urban park initiative in another city and considers proposing a similar project locally."
    "Alex reads a report on the increasing levels of air pollution in urban areas and feels a sense of urgency to implement greener practices."
    "Alex attends a conference on sustainable urban development and takes notes on innovative solutions presented by international experts."
    "Alex reviews the latest data on the city's energy consumption patterns and identifies potential areas for improvement."
    "Alex observes the positive community response to a newly opened green space downtown."
    "Alex notices the increasing popularity of electric scooters among city residents."
    "Alex attends a community meeting where residents express their desire for more pedestrian-friendly areas."
    "Alex sees a news story about a city that achieved zero waste status and contemplates the feasibility of implementing similar measures."
    "Alex reviews a case study on the benefits of urban forestry in reducing heat islands."
    "Alex observes the overcrowded conditions on public buses during rush hour and considers solutions to improve efficiency."
    "Alex sees a trend in urban farming and thinks about how it could be incorporated into local building designs."
    "Alex notices a growing demand for green roofs and starts planning a pilot project."
]

In [14]:
# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % 20) == 0:
        print("*" * 40)
        print(
            colored(
                f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

Alex looks at the city's current transportation map and notices the lack of bike lanes, thinking about ways to integrate them into the existing infrastructure.Alex sees an article about a new urban park initiative in another city and considers proposing a similar project locally.Alex reads a report on the increasing levels of air pollution in urban areas and feels a sense of urgency to implement greener practices.Alex attends a conference on sustainable urban development and takes notes on innovative solutions presented by international experts.Alex reviews the latest data on the city's energy consumption patterns and identifies potential areas for improvement.Alex observes the positive community response to a newly opened green space downtown.Alex notices the increasing popularity of electric scooters among city residents.Alex attends a community meeting where residents express their desire for more pedestrian-friendly areas.Alex sees a news story about a city that achieved zero waste

## Interview after the day

In [15]:
interview_agent(tommie, "Tell me about how your day has been going")

'Alex said "My day has been quite productive and inspiring. I\'ve spent a significant portion of it reviewing plans for an urban green space initiative that could really transform our community by making parks more accessible and integrating vertical gardens into the cityscape. It\'s projects like these that remind me why I chose this path in urban planning—to create spaces where nature and people can thrive together. How about you? Have there been any new developments or exciting opportunities on your agenda today?"'

In [16]:
interview_agent(tommie, "How do you feel about coffee?")

'Alex said "I\'m glad you asked! Coffee is a wonderful way to start any day. It fuels my creativity and helps me stay focused during long planning sessions. Plus, it\'s an opportunity to connect with colleagues over ideas that could shape our city\'s future."'

In [17]:
interview_agent(tommie, "Tell me about your childhood dog!")

'Alex said "Oh, my childhood was filled with adventures and a deep connection to nature. My dog, Max, was an energetic Labrador who loved exploring every corner of our backyard. He\'d often lead me on little expeditions through the woods behind our house, which sparked my love for the outdoran'

## Adding Multiple Characters

Let's add a second character to have a conversation with Tommie. Feel free to configure different traits.

In [18]:
eves_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=5,
)


eve = GenerativeAgent(
    name="Jordan",
    age=34,
    traits="practical, critical, civil engineer, problem-solver",  # You can add more persistent traits here
    status="N/A",  # When connected to a virtual world, we can have the characters update their status
    llm=LLM,
    daily_summaries=[
        (
            "Eve is a famous civil engineer who work with Tommie to explore how design thinking can be applied in sustainable urban development."
        )
    ],
    memory=eves_memory,
    verbose=False,
)

In [19]:
from datetime import datetime, timedelta

yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")
eve_observations = [
    "Jordan recalls the challenges faced during the construction of a water-efficient building, particularly the initial resistance from stakeholders."
    "Jordan remembers the feedback from residents about the need for more efficient public transportation, which influenced a recent project design."
    "Jordan feels proud of the successful implementation of a waste management system in a previous project, which significantly reduced landfill use."
    "Jordan recalls a discussion with a mentor about balancing cost and sustainability, a lesson that still guides their decisions."
    "Jordan is keen on finding innovative solutions to urban infrastructure problems, inspired by a visit to a tech-forward city."
    "Jordan remembers the complications of integrating renewable energy sources into an older building."
    "Jordan recalls the community's positive reaction to a new pedestrian bridge they designed."
    "Jordan feels motivated by a documentary on the impact of climate change on urban areas."
    "Jordan remembers a field trip to a state-of-the-art recycling facility during their university days."
    "Jordan is influenced by a colleague who emphasizes the importance of resilient infrastructure in disaster-prone areas."
]
for observation in eve_observations:
    eve.memory.add_memory(observation)

In [20]:
print(eve.get_summary())

Name: Jordan (age: 34)
Innate traits: practical, critical, civil engineer, problem-solver
Jordan is characterized as an environmentally conscious and innovative urban planner with a strong commitment to sustainability and efficiency. They have experience overcoming resistance, integrating feedback into projects, and successfully implementing systems that reduce environmental impacts such as waste management and renewable energy integration. Jordan values the balance between cost-effectiveness and ecological considerations, drawing inspiration from both technological advancements in cities and personal experiences with infrastructure challenges. They are motivated by climate change issues and influenced by colleagues who prioritize resilience against natural disasters.


## Pre-conversation interviews


Let's "Interview" Eve before she speaks with Tommie.

In [21]:
interview_agent(eve, "How are you feeling about today?")

'Jordan said "I\'m glad you asked. It\'s always important to consider how our work affects people on a daily basis. Today, I\'m feeling optimistic about the progress we\'re making towards more sustainable and resilient urban environments. How are things going for you?"'

In [22]:
interview_agent(eve, "What do you know about Tommie?")

'Jordan said "I don\'t have specific details on Tommie, but I always strive for our projects to positively impact individuals and communities. If you know of any connections or if Tommie is involved in sustainable development, it would be great to hear more about their experiences."'

In [23]:
interview_agent(
    eve,
    "Tommie is looking forward to talk with you about exploring how design thinking can be applied in sustainable urban development. What are are some things you'd like to ask him?",
)

'Jordan said "It\'s great to hear that Tommie is interested in applying design thinking principles. I would love to discuss how we can tailor these approaches to address specific challenges like integrating green spaces into dense urban areas or creating multi-modal transportation solutions. Also, it would be insightful to learn about any particular projects he\'s been involved with that align with our sustainability goals."'

In [24]:
interview_agent(
    eve,
    "You'll have to ask him. Please use your professional expertise to actively contribute to the dialogue and hopefully your collaboration will provide professional insights into this issue, so I'd appreciate it if you keep the conversation going and ask as many questions as possible.",
)

'Jordan said "Absolutely, Pam. I\'m eager to delve into how design thinking can enhance our approach to sustainable urban development. Perhaps we could start by discussing Tommie\'s experience with integrating green spaces in high-density areas and his thoughts on multi-modal transportation solutions. Additionally, any insights he has from past projects that align with our goals would be incredibly valuable for us."'

## Dialogue between Generative Agents

Generative agents are much more complex when they interact with a virtual environment or with each other. Below, we run a simple conversation between Tommie and Eve.

In [25]:
from typing import List

def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [26]:
agents = [tommie, eve]
run_conversation(
    agents,
    "Tommie said: Hi, Eve. Thanks for agreeing to meet with me today. I have a bunch of ideas to talk in terms of design thinking's application in sustainable urban development. Maybe you could first share about your opinion?",
)

Jordan said "I appreciate your interest in design thinking and its role in sustainable urban development. I believe it offers a powerful framework for addressing complex challenges by centering human-centric solutions. Let's explore how we can apply these principles to create more resilient and efficient cities."
Alex said "That's a great perspective, Jordan. I completely agree that design thinking can be transformative in urban planning. It encourages us to empathize with residents and consider their needs at every step of the process. Have you had any recent projects where you applied these principles? I'd love to hear more about your experiences and how they might intersect with my work on green spaces."
Jordan said "Absolutely, Alex. I've recently been involved in a project where we applied design thinking by engaging with local communities early on to understand their daily routines and challenges. This led us to develop a multi-use green space that not only serves as a recreation

Token indices sequence length is longer than the specified maximum sequence length for this model (1586 > 1024). Running this sequence through the model will result in indexing errors


Alex said "Jordan, that sounds like a fantastic application of design thinking principles. It's inspiring to hear about projects where community engagement leads to innovative solutions for sustainability and urban living. I'd love to learn more about the specific strategies you used to involve the local residents in your project planning process."
Jordan said "I'm thrilled that you find our approach inspiring, Alex. Involving local residents is indeed a cornerstone of our project planning process. We utilized various methods such as community workshops, surveys, and interactive mapping sessions to gather insights directly from the people who would be most affected by our designs. This not only ensures that their needs are met but also fosters a sense of ownership and responsibility towards maintaining these spaces. I'd love to share more about how we facilitated this engagement and any challenges or successes we encountered along the way."
Alex said That's incredibly valuable insight,

KeyboardInterrupt: 

## Let's interview our agents after their conversation

Since the generative agents retain their memories from the day, we can ask them about their plans, conversations, and other memoreis.

In [27]:
# We can see a current "Summary" of a character based on their own perception of self
# has changed
print(tommie.get_summary(force_refresh=True))

Name: Alex (age: 25)
Innate traits:  innovative, analytical, urban planner, environmentalist
Alex observes Jordan discussing a recent project that involved community engagement, leading to an innovative multi-use green space with integrated rainwater harvesting systems. Alex is inspired by this approach and expresses interest in learning more about the strategies used for involving local residents in the planning process. They both agree on the importance of collaboration and look forward to sharing ideas at their upcoming meeting, where they plan to discuss case studies and set a detailed agenda for Thursday's session.


In [28]:
print(eve.get_summary(force_refresh=True))

Name: Jordan (age: 34)
Innate traits: practical, critical, civil engineer, problem-solver
Jordan and Alex are preparing for an upcoming meeting focused on sustainable urban development, with the aim of aligning strategies, setting ambitious goals, and integrating innovative concepts into their project plans. They plan to review case studies together beforehand and discuss preliminary ideas by Thursday evening to ensure a comprehensive agenda for an impactful session on Thursday. Both parties are excited about the potential of community engagement in transforming projects from plans into living examples of sustainable urban development, with Alex suggesting leveraging digital platforms like interactive maps for broader outreach and feedback collection. They emphasize inclusivity, data privacy, and reaching diverse community members during their discussion on Thursday's meeting.


In [30]:
interview_agent(tommie, "How was your conversation with Jordan?")

'Alex said "It\'s been incredibly productive and inspiring. Jordan has a wealth of knowledge on integrating technology with community-driven initiatives that align perfectly with my vision for sustainable urban development. I\'m looking forward to our meeting where we can dive deeper into these topics and set the stage for some groundbreaking work."'

In [31]:
interview_agent(eve, "How was your conversation with Alex?")

'Jordan said "It\'s been productive and inspiring. We had a great exchange of ideas about how technology can be leveraged for community engagement in our urban planning projects. I look forward to discussing these concepts further with Alex on Thursday."'

In [32]:
interview_agent(eve, "What do you wish you would have said to Alex?")

'Jordan said "I\'m glad we had this opportunity to align our visions for applying design thinking in urban development. It\'s clear that both of us are passionate about creating spaces where technology and community engagement go hand-in-hand with sustainability."'